In [1]:
import nbformat as nbf

# 노트북 객체 생성
nb = nbf.v4.new_notebook()

# -----------------------------------------------------------------------------
# 1. 제목 및 설정
# -----------------------------------------------------------------------------
text_header = """\
# Korean Economic Data Forecasting (Python Version)
이 노트북은 원본 R 코드(KRM_rcode_part1.R, part2.R)를 Python으로 포팅한 버전입니다.
한국 경제 데이터(CPI)를 예측하기 위해 Rolling Window 방식을 사용하여 다양한 모델(AR, Lasso, RF, XGBoost, LSTM)을 비교합니다.

**필요 라이브러리:**
* `pandas`, `numpy`, `matplotlib`, `seaborn`
* `scikit-learn`, `statsmodels`, `xgboost`
* `tensorflow` (Keras)
* `boruta` (pip install boruta)
"""

code_imports = """\
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Modeling
from sklearn.linear_model import Lasso, ElasticNet, LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from xgboost import XGBRegressor

# Time Series
from statsmodels.tsa.ar_model import AutoReg

# Deep Learning
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

# Feature Selection
from boruta import BorutaPy

# Warning 무시
import warnings
warnings.filterwarnings('ignore')

print("Libraries loaded.")
"""

# -----------------------------------------------------------------------------
# 2. 유틸리티 함수 (R의 사용자 정의 함수 대체)
# -----------------------------------------------------------------------------
text_utils = """\
## 1. Utility Functions
R 코드에서 `source()`로 불러오던 롤링 윈도우(Rolling Window) 및 데이터 변환 함수들을 Python으로 구현합니다.
"""

code_utils = """\
def create_lagged_features(data, lag=1):
    \"\"\"
    시계열 데이터를 지도학습용(X, y) 데이터로 변환 (R의 embed 함수 역할)
    \"\"\"
    df = pd.DataFrame(data)
    columns = [df.shift(i) for i in range(lag + 1)]
    df_lagged = pd.concat(columns, axis=1)
    df_lagged.columns = ['t'] + [f't-{i}' for i in range(1, lag + 1)]
    df_lagged = df_lagged.dropna()
    
    y = df_lagged.iloc[:, 0].values
    X = df_lagged.iloc[:, 1:].values
    return X, y

def rolling_window_forecast(X, y, model_type, npred, horizon=1, params=None):
    \"\"\"
    R의 *.rolling.window 함수들을 대체하는 통합 함수
    \"\"\"
    predictions = []
    real_values = []
    
    # 전체 데이터 길이
    T = len(y)
    # 학습 시작 인덱스 (총 길이 - 예측 횟수)
    start_idx = T - npred
    
    print(f"Running {model_type} (h={horizon})...")
    
    for i in range(npred):
        # Rolling Window: 현재 시점 t까지의 데이터를 사용하여 t+horizon 예측
        # Train data: start_idx + i 시점 이전까지
        curr_idx = start_idx + i
        
        # Horizon 고려: h=1이면 바로 다음달, h=12면 12달 뒤 예측이므로
        # 학습 데이터는 타겟 시점(curr_idx)보다 horizon만큼 이전 데이터까지만 사용 가능해야 함
        # 편의상 여기서는 Direct Multi-step 방식(데이터 셋 자체를 h시점 밀어서 구성)을 가정하거나
        # 반복적인 1-step 예측이 아닌, 해당 시점 모델 재학습 구조로 작성
        
        X_train = X[:curr_idx-horizon]
        y_train = y[:curr_idx-horizon]
        
        # Test input (가장 최근 관측치)
        X_test = X[curr_idx-horizon:curr_idx-horizon+1] 
        actual = y[curr_idx] # 실제값
        
        if len(y_train) < 10: # 데이터가 너무 적으면 건너뜀
            predictions.append(np.nan)
            real_values.append(actual)
            continue

        pred = 0
        
        try:
            if model_type == 'RW': # Random Walk
                pred = y_train[-1] # 직전 관측치 그대로 예측
                
            elif model_type == 'AR':
                # Statsmodels AutoReg
                model = AutoReg(y_train, lags=params.get('lags', 1)).fit()
                pred = model.predict(start=len(y_train), end=len(y_train) + horizon - 1)[-1]
                
            elif model_type in ['Lasso', 'ElasticNet', 'RF', 'XGB']:
                if model_type == 'Lasso':
                    model = Lasso(alpha=params.get('alpha', 1.0))
                elif model_type == 'ElasticNet':
                    model = ElasticNet(alpha=params.get('alpha', 0.5), l1_ratio=0.5)
                elif model_type == 'RF':
                    model = RandomForestRegressor(n_estimators=100, random_state=42)
                elif model_type == 'XGB':
                    model = XGBRegressor(n_estimators=100, objective='reg:squarederror', verbosity=0)
                
                model.fit(X_train, y_train)
                pred = model.predict(X_test)[0]
                
            elif model_type == 'NN': # Simple MLP
                tf.random.set_seed(42)
                model = Sequential()
                model.add(Dense(32, activation='relu', input_dim=X_train.shape[1]))
                model.add(Dense(16, activation='relu'))
                model.add(Dense(1))
                model.compile(optimizer='adam', loss='mse')
                model.fit(X_train, y_train, epochs=10, verbose=0, batch_size=16)
                pred = model.predict(X_test, verbose=0)[0][0]
                
        except Exception as e:
            # 에러 발생 시 Na
            pred = np.nan
            
        predictions.append(pred)
        real_values.append(actual)
        
    return np.array(predictions), np.array(real_values)
"""

# -----------------------------------------------------------------------------
# 3. Part 1 데이터 로드 및 변환
# -----------------------------------------------------------------------------
text_part1_data = """\
## 2. Part 1: Data Loading & Standard Transformation
R 코드의 1-7번 변환 코드를 적용합니다.
"""

code_part1_data = """\
# 데이터 로드 (경로를 실제 환경에 맞게 수정하세요)
file_path = "project2024/DataKoreaFrom200408To202306.csv"

# 파일이 없으면 더미 데이터 생성 (테스트용)
if not os.path.exists(file_path):
    print("Warning: File not found. Creating dummy data for demonstration.")
    dates = pd.date_range(start='2004-08-01', periods=227, freq='M')
    data_raw = pd.DataFrame(np.random.randn(227, 95) + 100, index=dates)
    data_raw.columns = ['Code'] + [f'Var{i}' for i in range(1, 95)]
    # 첫 행은 Transformation Code (1,2,4,5,6,7 임의 할당)
    tcode_row = np.random.choice([1, 2, 4, 5, 6, 7], 95)
    data_raw.iloc[0, :] = tcode_row
    data_raw.columns = ['Date'] + [f'V{i}' for i in range(1, 95)] # Column 이름 재설정 가정
    data_raw.iloc[0, 65] = 1 # CPI 가정
else:
    data_raw = pd.read_csv(file_path)

# Transformation Code 추출
tcode = data_raw.iloc[0, :].values
data = data_raw.iloc[1:, :].apply(pd.to_numeric) # Code 행 제거 및 숫자 변환

# 데이터프레임 초기화 (앞 2개 행 제거 예정이므로)
tdata = data.iloc[2:].copy()
tdata[:] = np.nan

# R 코드의 Loop 로직 (2:94) -> Python (1:94) (인덱스 주의)
# R: tcode[i] == 1 -> No transform
# R: tcode[i] == 2 -> diff(x)
# R: tcode[i] == 4 -> log(x)
# R: tcode[i] == 5 -> diff(log(x))
# R: tcode[i] == 6 -> diff(diff(log(x)))
# R: tcode[i] == 7 -> diff(x[t]/x[t-1]) (Growth rate diff)

for i in range(1, 94): # Column index assuming 0 is Date or similar
    col_name = data.columns[i]
    code = tcode[i]
    
    series = data.iloc[:, i].astype(float)
    
    if code == 1:
        tdata.iloc[:, i] = series.iloc[2:].values
    elif code == 2:
        # 1차 차분 (앞 2개 행 제거된 길이에 맞춰야 함)
        temp = series.diff().iloc[2:] 
        tdata.iloc[:, i] = temp.values
    elif code == 4:
        tdata.iloc[:, i] = np.log(series).iloc[2:].values
    elif code == 5:
        # Log Differencing
        temp = np.log(series).diff().iloc[2:]
        tdata.iloc[:, i] = temp.values
    elif code == 6:
        # Log -> Diff -> Diff
        temp = np.log(series).diff().diff().iloc[2:]
        tdata.iloc[:, i] = temp.values
    elif code == 7:
        # 증가율의 차분
        growth = series / series.shift(1)
        temp = growth.diff().iloc[2:]
        tdata.iloc[:, i] = temp.values

# 결측치 제거
fdata = tdata.dropna(axis=1, how='all').dropna(axis=0)

# CPI 컬럼 찾기 (R 코드에서는 65번째 열)
# 여기서는 'CPI'라는 이름이나 65번째 인덱스를 사용
target_col_idx = 64 # Python 0-based index (65th)
target_col_name = fdata.columns[target_col_idx]
print(f"Target Variable: {target_col_name}")

# Y 매트릭스 구성 (CPI가 첫번째 열로 오도록)
cols = [target_col_name] + [c for c in fdata.columns if c != target_col_name]
Y_df = fdata[cols]
Y = Y_df.values

print("Data shape:", Y.shape)
"""

# -----------------------------------------------------------------------------
# 4. Part 1 모델링 실행
# -----------------------------------------------------------------------------
text_part1_models = """\
## 3. Part 1: Rolling Window Forecasting
RW, AR, Lasso, RF, XGB, NN 모델을 실행합니다.
"""

code_part1_models = """\
npred = 105
horizons = [1, 3, 6, 12]
results = {}

# 데이터 준비 (Lagged Features 생성)
# 모든 모델이 Lagged Feature를 쓴다고 가정 (ARIMA/RW 제외)
# 편의상 lag=12 (최대 horizon)로 고정하여 데이터셋 생성
lag_max = 12
X_feat, y_target = create_lagged_features(Y_df[target_col_name], lag=lag_max)

# --- Loop over Horizons ---
for h in horizons:
    print(f"\\n=== Horizon: {h} ===")
    
    # 1. Random Walk
    # RW는 Feature 불필요, 단순 y값 사용
    pred_rw, real_rw = rolling_window_forecast(X_feat, y_target, 'RW', npred, h)
    results[f'RW_h{h}'] = mean_squared_error(real_rw, pred_rw, squared=False) # RMSE
    
    # 2. AR (AutoReg)
    pred_ar, _ = rolling_window_forecast(X_feat, y_target, 'AR', npred, h, {'lags': 4})
    results[f'AR_h{h}'] = mean_squared_error(real_rw, pred_ar, squared=False)
    
    # 3. Lasso
    pred_lasso, _ = rolling_window_forecast(X_feat, y_target, 'Lasso', npred, h, {'alpha': 0.01})
    results[f'Lasso_h{h}'] = mean_squared_error(real_rw, pred_lasso, squared=False)
    
    # 4. Random Forest
    pred_rf, _ = rolling_window_forecast(X_feat, y_target, 'RF', npred, h)
    results[f'RF_h{h}'] = mean_squared_error(real_rw, pred_rf, squared=False)
    
    # 5. XGBoost
    pred_xgb, _ = rolling_window_forecast(X_feat, y_target, 'XGB', npred, h)
    results[f'XGB_h{h}'] = mean_squared_error(real_rw, pred_xgb, squared=False)

# 결과 출력
res_df = pd.DataFrame(list(results.items()), columns=['Model_Horizon', 'RMSE'])
print(res_df)
"""

# -----------------------------------------------------------------------------
# 5. Boruta Feature Selection
# -----------------------------------------------------------------------------
text_boruta = """\
## 4. Boruta Feature Selection
변수 중요도를 산출하여 상위 변수만 선택합니다.
"""

code_boruta = """\
# Boruta를 위한 RF 설정
rf = RandomForestRegressor(n_jobs=-1, max_depth=5)

# Boruta 실행 (Lag=12 데이터 사용)
# X_feat는 앞서 생성한 Lagged Feature Matrix
feat_selector = BorutaPy(rf, n_estimators='auto', verbose=0, random_state=42)

# Boruta는 Numpy array 입력 필요
print("Starting Boruta...")
feat_selector.fit(X_feat, y_target)

# 선택된 변수 확인
selected_indices = np.where(feat_selector.support_)[0]
print(f"Number of selected features: {len(selected_indices)}")

# 선택된 변수만 사용하여 모델 재학습 (BS_RF)
X_selected = X_feat[:, selected_indices]

# BS_RF 실행 (Horizon 1 예시)
pred_bs_rf, real = rolling_window_forecast(X_selected, y_target, 'RF', npred, 1)
rmse_bs_rf = mean_squared_error(real, pred_bs_rf, squared=False)
print(f"Boruta Selected RF (h=1) RMSE: {rmse_bs_rf}")
"""

# -----------------------------------------------------------------------------
# 6. Part 2: YoY 데이터 변환 및 모델링
# -----------------------------------------------------------------------------
text_part2 = """\
## 5. Part 2: YoY Transformation Analysis
CPI 데이터를 전년 동월 대비 로그 차분($\\ln(CPI_t) - \\ln(CPI_{t-12})$)하여 예측합니다.
"""

code_part2 = """\
# 원본 데이터 다시 로드
data_p2 = data_raw.iloc[1:, :].apply(pd.to_numeric)
cpi_series = data_p2.iloc[:, 64] # CPI Index

# YoY Log Difference 계산
cpi_log = np.log(cpi_series)
cpi_yoy = cpi_log.diff(12).dropna()

print(f"YoY Data Length: {len(cpi_yoy)}")

# Part 2용 타겟 데이터 생성
y_target_p2 = cpi_yoy.values
X_feat_p2, y_target_p2 = create_lagged_features(y_target_p2, lag=12)

# 예측 수행 (예시: XGBoost Horizon 1)
npred_p2 = 93
pred_p2, real_p2 = rolling_window_forecast(X_feat_p2, y_target_p2, 'XGB', npred_p2, 1)

rmse_p2 = mean_squared_error(real_p2, pred_p2, squared=False)
print(f"Part 2 (YoY) XGBoost RMSE: {rmse_p2}")
"""

text_export = """\
## 6. Result Export
결과를 Excel 등으로 저장합니다.
"""
code_export = """\
# res_df.to_excel("forecast_errors.xlsx")
print("Analysis Complete.")
"""

# -----------------------------------------------------------------------------
# 셀 추가
# -----------------------------------------------------------------------------
nb['cells'] = [
    nbf.v4.new_markdown_cell(text_header),
    nbf.v4.new_code_cell(code_imports),
    nbf.v4.new_markdown_cell(text_utils),
    nbf.v4.new_code_cell(code_utils),
    nbf.v4.new_markdown_cell(text_part1_data),
    nbf.v4.new_code_cell(code_part1_data),
    nbf.v4.new_markdown_cell(text_part1_models),
    nbf.v4.new_code_cell(code_part1_models),
    nbf.v4.new_markdown_cell(text_boruta),
    nbf.v4.new_code_cell(code_boruta),
    nbf.v4.new_markdown_cell(text_part2),
    nbf.v4.new_code_cell(code_part2),
    nbf.v4.new_markdown_cell(text_export),
    nbf.v4.new_code_cell(code_export)
]

# -----------------------------------------------------------------------------
# 파일 저장
# -----------------------------------------------------------------------------
with open('KRM_Project_Python.ipynb', 'w', encoding='utf-8') as f:
    nbf.write(nb, f)

print("KRM_Project_Python.ipynb 파일이 생성되었습니다.")

KRM_Project_Python.ipynb 파일이 생성되었습니다.
